In [1]:
import pandas as pd 
import numpy as np 
from tkinter import * 
import ttkbootstrap as tb
from tkinter import messagebox, ttk, filedialog, colorchooser
from PIL import ImageTk, Image
import os
import pathlib
import datetime as dt
import csv 
import pyodbc
import urllib
from sqlalchemy import create_engine

In [2]:
server = 'ADMIN-PC'
database = 'HSX_thong_ke'

quoted = urllib.parse.quote_plus('Driver={SQL Server};'
                                 'Server='+server+';'
                                 'Database='+database+';')
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server='+server+';'
                      'Database='+database+';'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [3]:
df_hsx_index = pd.read_csv('hsx_index.csv')
df_hsx_index['ETL_date'] = str(dt.datetime.now())
df_hsx_index

,Mã,None 202107,None 202212,Vn30 202107,Vn30 202212,VnMid 202107,VnMid 202212,VnSmall 202107,VnSmall 202212,ETL_date
0,AAA,0,0,0,0,202107,202212,0,0,2025-09-05 07:34:28.175371
1,AAM,202107,0,0,0,0,0,0,202212,2025-09-05 07:34:28.175371
2,AAT,202107,0,0,0,0,0,0,202212,2025-09-05 07:34:28.175371
3,ABR,202107,0,0,0,0,0,0,202212,2025-09-05 07:34:28.175371
4,ABS,0,0,0,0,0,0,202107,202212,2025-09-05 07:34:28.175371
...,...,...,...,...,...,...,...,...,...,...
441,VSI,0,202212,0,0,0,0,202107,0,2025-09-05 07:34:28.175371
442,VTB,0,0,0,0,0,0,202107,202212,2025-09-05 07:34:28.175371
443,VTO,0,0,0,0,0,0,202107,202212,2025-09-05 07:34:28.175371
444,YBM,0,0,0,0,0,0,202107,202212,2025-09-05 07:34:28.175371


In [4]:
query = '''if (OBJECT_ID('dim_hsx_index') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_hsx_index
	(
		[Ma_ck] varchar(10),
		[None_202107] int,
        [None_202212] int,
        [Vn30_202107] int,
        [Vn30_202212] int,
        [Vnmid_202107] int,
        [Vnmid_202212] int,
        [Vnsmall_202107] int,
        [Vnsmall_202212] int,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

df_hsx_index.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_hsx_index.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_hsx_index.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_hsx_index')
    cursor.executemany('''INSERT INTO dim_hsx_index Values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', reader)
    conn.commit()

In [5]:
df_gdp = pd.read_excel('gdp.xlsx')
df_gdp['ETL_date'] = str(dt.datetime.now())
df_gdp['GDP'] = df_gdp['GDP'].astype('float')
df_gdp

,Date,GDP,ETL_date
0,2019-03-31,1116680.0,2025-09-05 07:34:28.839982
1,2019-06-30,1366899.0,2025-09-05 07:34:28.839982
2,2019-09-30,1539114.0,2025-09-05 07:34:28.839982
3,2019-12-31,2010887.0,2025-09-05 07:34:28.839982
4,2020-03-31,1188207.0,2025-09-05 07:34:28.839982
5,2020-06-30,1382995.0,2025-09-05 07:34:28.839982
6,2020-09-30,1593586.0,2025-09-05 07:34:28.839982
7,2020-12-31,2121415.0,2025-09-05 07:34:28.839982
8,2021-03-31,1915368.0,2025-09-05 07:34:28.839982
9,2021-06-30,2050431.0,2025-09-05 07:34:28.839982


In [6]:
query = '''if (OBJECT_ID('fact_gdp') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_gdp
	(
		[Date] datetime2,
		[gdp_ty_vnd] float,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

df_gdp.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_gdp.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_gdp.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_gdp')
    cursor.executemany('''INSERT INTO fact_gdp Values (?, ?, ?)''', reader)
    conn.commit()

In [ ]:
lst_nam = ["2019", "2020", "2021", "2022"]
lst_thang = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]
lst_path_text = []
for i in lst_nam:
    for j in lst_thang:
        lst_path_text.append(f'D:\\Tin hoc\\Project\\Thong ke thi truong\\Market Update\\HSX\\Raw HSX\\Chi tiet giao dich theo loai NDT (VNI)\\{i}\\Chi tiet GDTLNDT {i}\\Thang {j}')
lst_path_text = [pathlib.Path(i) for i in lst_path_text]
lst_path_text

files = []
for path in lst_path_text:
    for direc in path.iterdir():
        file = direc.stem
        if not file.startswith("G"):
            files.append(direc)

lst_df = []
for path in files:
    date = path.stem

    skiprows = [i for i in range(0,11)]
    df = pd.read_excel(path, skiprows=lambda x: x in skiprows, sheet_name='ExportData')
    df = df.iloc[:, :-1]
    lst_phan_nhom = ['Cá nhân trong nước', 'Cá nhân nước ngoài', 'Tổ chức trong nước', 'Tổ chức nước ngoài']
    lst_phan_nhom = [[i]*6 for i in lst_phan_nhom]
    lst_phan_nhom = lst_phan_nhom[0] + lst_phan_nhom[1] + lst_phan_nhom[2] + lst_phan_nhom[3]

    lst_abc_phan_loai = list(df.iloc[0].values[1:][0:6])
    lst_abc_phan_loai = lst_abc_phan_loai * 4

    lst_column1 = list(zip(lst_phan_nhom, lst_abc_phan_loai))
    lst_column2 = []
    for i in lst_column1:
        lst_column2.append(i[0] + " - " + i[1])

    lst_column2.insert(0, "Ma_ck")
    df.columns = lst_column2
    df = df.iloc[4:]
    df = df.reset_index(drop=True)
    df["Ngay"] = dt.datetime.strptime(date, "%m.%d.%Y")
    lst_df.append(df)
df_new = pd.DataFrame(pd.concat(lst_df))
df_new = df_new[df_new["Cá nhân trong nước - Khối lượng mua (CP)"].isnull() == False]
df_new = df_new.reset_index(drop=True)
df_new.head(5)

###########################################

df_gtm = df_new[['Ma_ck', 'Ngay', 'Cá nhân trong nước - Giá trị mua  (Nghìn VNĐ)', 'Cá nhân nước ngoài - Giá trị mua  (Nghìn VNĐ)',
               'Tổ chức trong nước - Giá trị mua  (Nghìn VNĐ)', 'Tổ chức nước ngoài - Giá trị mua  (Nghìn VNĐ)']].copy()
df_gtm['Cá nhân trong nước - Giá trị mua  (Nghìn VNĐ)'] = df_gtm['Cá nhân trong nước - Giá trị mua  (Nghìn VNĐ)'].astype('float')
df_gtm['Cá nhân nước ngoài - Giá trị mua  (Nghìn VNĐ)'] = df_gtm['Cá nhân nước ngoài - Giá trị mua  (Nghìn VNĐ)'].astype('float')
df_gtm['Tổ chức trong nước - Giá trị mua  (Nghìn VNĐ)'] = df_gtm['Tổ chức trong nước - Giá trị mua  (Nghìn VNĐ)'].astype('float')
df_gtm['Tổ chức nước ngoài - Giá trị mua  (Nghìn VNĐ)'] = df_gtm['Tổ chức nước ngoài - Giá trị mua  (Nghìn VNĐ)'].astype('float')
df_gtm = df_gtm.melt(id_vars=["Ma_ck", "Ngay"], var_name="Phan_loai", value_name="Value")
df_gtm["index"] = df_gtm["Phan_loai"].str.index('-')
df_gtm["Phan_loai"] = df_gtm.apply(lambda row: row["Phan_loai"][:row["index"]], axis=1)
df_gtm["Phan_loai"] = df_gtm["Phan_loai"].str.strip()
df_gtm = df_gtm[["Ma_ck", "Ngay", "Phan_loai", "Value"]]
df_gtm['ETL_date'] = str(dt.datetime.now())

######################################################

query = '''if (OBJECT_ID('fact_gtm') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_gtm
	(
		[Ma_ck] varchar(10),
		[Ngay] datetime2,
        [Phan_loai] nvarchar(255),
        [Value_nghin_vnd] float,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

##################################################

df_gtm.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_gtm.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_gtm.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_gtm')
    cursor.executemany('''INSERT INTO fact_gtm Values (?, ?, ?, ?, ?)''', reader)
    conn.commit()

##################################################

df_klm = df_new[['Ma_ck', 'Ngay', 'Cá nhân trong nước - Khối lượng mua (CP)', 'Cá nhân nước ngoài - Khối lượng mua (CP)', 'Tổ chức trong nước - Khối lượng mua (CP)',
                 'Tổ chức nước ngoài - Khối lượng mua (CP)']].copy()
df_klm['Cá nhân trong nước - Khối lượng mua (CP)'] = df_klm['Cá nhân trong nước - Khối lượng mua (CP)'].astype('float')
df_klm['Cá nhân nước ngoài - Khối lượng mua (CP)'] = df_klm['Cá nhân nước ngoài - Khối lượng mua (CP)'].astype('float')
df_klm['Tổ chức trong nước - Khối lượng mua (CP)'] = df_klm['Tổ chức trong nước - Khối lượng mua (CP)'].astype('float')
df_klm['Tổ chức nước ngoài - Khối lượng mua (CP)'] = df_klm['Tổ chức nước ngoài - Khối lượng mua (CP)'].astype('float')
df_klm = df_klm.melt(id_vars=["Ma_ck", "Ngay"], var_name="Phan_loai", value_name="Value")
df_klm["index"] = df_klm["Phan_loai"].str.index('-')
df_klm["Phan_loai"] = df_klm.apply(lambda row: row["Phan_loai"][:row["index"]], axis=1)
df_klm["Phan_loai"] = df_klm["Phan_loai"].str.strip()
df_klm = df_klm[["Ma_ck", "Ngay", "Phan_loai", "Value"]]
df_klm['ETL_date'] = str(dt.datetime.now())
df_klm

########################################################

query = '''if (OBJECT_ID('fact_klm') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_klm
	(
		[Ma_ck] varchar(10),
		[Ngay] datetime2,
        [Phan_loai] nvarchar(255),
        [Value_cp] float,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

##########################################################

df_klm.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_klm.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_klm.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_klm')
    cursor.executemany(''' INSERT INTO fact_klm Values (?, ?, ?, ?, ?)''', reader)
    conn.commit()

###########################################################

df_gtb = df_new[['Ma_ck', 'Ngay', 'Cá nhân trong nước - Giá trị Bán (Nghìn VNĐ)', 'Cá nhân nước ngoài - Giá trị Bán (Nghìn VNĐ)', 'Tổ chức trong nước - Giá trị Bán (Nghìn VNĐ)',
                 'Tổ chức nước ngoài - Giá trị Bán (Nghìn VNĐ)']].copy()
df_gtb['Cá nhân trong nước - Giá trị Bán (Nghìn VNĐ)'] = df_gtb['Cá nhân trong nước - Giá trị Bán (Nghìn VNĐ)'].astype('float')
df_gtb['Cá nhân nước ngoài - Giá trị Bán (Nghìn VNĐ)'] = df_gtb['Cá nhân nước ngoài - Giá trị Bán (Nghìn VNĐ)'].astype('float')
df_gtb['Tổ chức trong nước - Giá trị Bán (Nghìn VNĐ)'] = df_gtb['Tổ chức trong nước - Giá trị Bán (Nghìn VNĐ)'].astype('float')
df_gtb['Tổ chức nước ngoài - Giá trị Bán (Nghìn VNĐ)'] = df_gtb['Tổ chức nước ngoài - Giá trị Bán (Nghìn VNĐ)'].astype('float')

df_gtb = df_gtb.melt(id_vars=["Ma_ck", "Ngay"], var_name="Phan_loai", value_name="Value")
df_gtb["index"] = df_gtb["Phan_loai"].str.index('-')
df_gtb["Phan_loai"] = df_gtb.apply(lambda row: row["Phan_loai"][:row["index"]], axis=1)
df_gtb["Phan_loai"] = df_gtb["Phan_loai"].str.strip()
df_gtb = df_gtb[["Ma_ck", "Ngay", "Phan_loai", "Value"]]
df_gtb['ETL_date'] = str(dt.datetime.now())
df_gtb

################################################################

query = '''if (OBJECT_ID('fact_gtb') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_gtb
	(
		[Ma_ck] varchar(10),
		[Ngay] datetime2,
        [Phan_loai] nvarchar(255),
        [Value_nghin_vnd] float,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

##############################################################################

df_gtb.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_gtb.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_gtb.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_gtb')
    cursor.executemany(''' INSERT INTO fact_gtb Values (?, ?, ?, ?, ?)''', reader)
    conn.commit()

##################################################################################

df_klb = df_new[['Ma_ck', 'Ngay', 'Cá nhân trong nước - Khối lượng Bán (CP)', 'Cá nhân nước ngoài - Khối lượng Bán (CP)', 'Tổ chức trong nước - Khối lượng Bán (CP)',
                  'Tổ chức nước ngoài - Khối lượng Bán (CP)']].copy()
df_klb['Cá nhân trong nước - Khối lượng Bán (CP)'] = df_klb['Cá nhân trong nước - Khối lượng Bán (CP)'].astype('float')
df_klb['Cá nhân nước ngoài - Khối lượng Bán (CP)'] = df_klb['Cá nhân nước ngoài - Khối lượng Bán (CP)'].astype('float')
df_klb['Tổ chức trong nước - Khối lượng Bán (CP)'] = df_klb['Tổ chức trong nước - Khối lượng Bán (CP)'].astype('float')
df_klb['Tổ chức nước ngoài - Khối lượng Bán (CP)'] = df_klb['Tổ chức nước ngoài - Khối lượng Bán (CP)'].astype('float')

df_klb = df_klb.melt(id_vars=["Ma_ck", "Ngay"], var_name="Phan_loai", value_name="Value")
df_klb["index"] = df_klb["Phan_loai"].str.index('-')
df_klb["Phan_loai"] = df_klb.apply(lambda row: row["Phan_loai"][:row["index"]], axis=1)
df_klb["Phan_loai"] = df_klb["Phan_loai"].str.strip()
df_klb = df_klb[["Ma_ck", "Ngay", "Phan_loai", "Value"]]
df_klb['ETL_date'] = str(dt.datetime.now())
df_klb

####################################################################################

query = '''if (OBJECT_ID('fact_klb') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_klb
	(
		[Ma_ck] varchar(10),
		[Ngay] datetime2,
        [Phan_loai] nvarchar(255),
        [Value_cp] float,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

######################################

df_klb.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_klb.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_klb.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_klb')
    cursor.executemany(''' INSERT INTO fact_klb Values (?, ?, ?, ?, ?)''', reader)
    conn.commit()

#########################################

df_gtr = df_new[['Ma_ck', 'Ngay', 'Cá nhân trong nước - Giá trị ròng (Nghìn VNĐ)', 'Cá nhân nước ngoài - Giá trị ròng (Nghìn VNĐ)', 
                  'Tổ chức trong nước - Giá trị ròng (Nghìn VNĐ)', 'Tổ chức nước ngoài - Giá trị ròng (Nghìn VNĐ)']].copy()

df_gtr['Cá nhân trong nước - Giá trị ròng (Nghìn VNĐ)'] = df_gtr['Cá nhân trong nước - Giá trị ròng (Nghìn VNĐ)'].astype('float')
df_gtr['Cá nhân nước ngoài - Giá trị ròng (Nghìn VNĐ)'] = df_gtr['Cá nhân nước ngoài - Giá trị ròng (Nghìn VNĐ)'].astype('float')
df_gtr['Tổ chức trong nước - Giá trị ròng (Nghìn VNĐ)'] = df_gtr['Tổ chức trong nước - Giá trị ròng (Nghìn VNĐ)'].astype('float')
df_gtr['Tổ chức nước ngoài - Giá trị ròng (Nghìn VNĐ)'] = df_gtr['Tổ chức nước ngoài - Giá trị ròng (Nghìn VNĐ)'].astype('float')

df_gtr = df_gtr.melt(id_vars=["Ma_ck", "Ngay"], var_name="Phan_loai", value_name="Value")
df_gtr["index"] = df_gtr["Phan_loai"].str.index('-')
df_gtr["Phan_loai"] = df_gtr.apply(lambda row: row["Phan_loai"][:row["index"]], axis=1)
df_gtr["Phan_loai"] = df_gtr["Phan_loai"].str.strip()
df_gtr = df_gtr[["Ma_ck", "Ngay", "Phan_loai", "Value"]]
df_gtr['ETL_date'] = str(dt.datetime.now())
df_gtr

############################################

query = '''if (OBJECT_ID('fact_gtr') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_gtr
	(
		[Ma_ck] varchar(10),
		[Ngay] datetime2,
        [Phan_loai] nvarchar(255),
        [Value_nghin_vnd] float,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

#############################################3

df_gtr.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_gtr.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_gtr.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_gtr')
    cursor.executemany(''' INSERT INTO fact_gtr Values (?, ?, ?, ?, ?)''', reader)
    conn.commit()

###############################

df_klr = df_new[['Ma_ck', 'Ngay', 'Cá nhân trong nước - Khối lượng ròng (CP)', 'Cá nhân nước ngoài - Khối lượng ròng (CP)', 'Tổ chức trong nước - Khối lượng ròng (CP)',
                  'Tổ chức nước ngoài - Khối lượng ròng (CP)']].copy()
df_klr['Cá nhân trong nước - Khối lượng ròng (CP)'] = df_klr['Cá nhân trong nước - Khối lượng ròng (CP)'].astype('float')
df_klr['Cá nhân nước ngoài - Khối lượng ròng (CP)'] = df_klr['Cá nhân nước ngoài - Khối lượng ròng (CP)'].astype('float')
df_klr['Tổ chức trong nước - Khối lượng ròng (CP)'] = df_klr['Tổ chức trong nước - Khối lượng ròng (CP)'].astype('float')
df_klr['Tổ chức nước ngoài - Khối lượng ròng (CP)'] = df_klr['Tổ chức nước ngoài - Khối lượng ròng (CP)'].astype('float')

df_klr = df_klr.melt(id_vars=["Ma_ck", "Ngay"], var_name="Phan_loai", value_name="Value")
df_klr["index"] = df_klr["Phan_loai"].str.index('-')
df_klr["Phan_loai"] = df_klr.apply(lambda row: row["Phan_loai"][:row["index"]], axis=1)
df_klr["Phan_loai"] = df_klr["Phan_loai"].str.strip()
df_klr = df_klr[["Ma_ck", "Ngay", "Phan_loai", "Value"]]
df_klr['ETL_date'] = str(dt.datetime.now())
df_klr

###################################

query = '''if (OBJECT_ID('fact_klr') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_klr
	(
		[Ma_ck] varchar(10),
		[Ngay] datetime2,
        [Phan_loai] nvarchar(255),
        [Value_cp] float,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

####################################

df_klr.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_klr.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_klr.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_klr')
    cursor.executemany(''' INSERT INTO fact_klr Values (?, ?, ?, ?, ?)''', reader)
    conn.commit()
    

In [8]:
# ETL_data_2 

lst_nam = ["2019", "2020", "2021", "2022"]
lst_thang = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]
lst_path_text = []
for i in lst_nam:
    for j in lst_thang:
        lst_path_text.append(f'D:\\Tin hoc\\Project\\Thong ke thi truong\\Market Update\\HSX\\Raw HSX\\Chi tiet Tu doanh\\{i}\\Thang {j}')
lst_path_text = [pathlib.Path(i) for i in lst_path_text]

files = []
for path in lst_path_text:
    for direc in path.iterdir():
        file = direc.stem
        if not file.startswith("G"):
            files.append(direc)

lst_df = []
for path in files:
    skiprows = [i for i in range(0,11)]
    date_name = path.stem
    df = pd.read_excel(path, sheet_name='ExportData', skiprows=lambda x: x in skiprows)
    df = df.iloc[:, :-2]
    df = df.iloc[2:]
    df = df.reset_index(drop=True)
    df["Ngay"] = dt.datetime.strptime(date_name, "%m.%d.%Y")
    lst_df.append(df)

df_new = pd.DataFrame(pd.concat(lst_df))
df_new = df_new[(df_new["Khối lượng mua"].isnull() == False) & (df_new["Khối lượng bán"].isnull() == False)]
df_new = df_new.reset_index(drop=True)
df_new["phan_loai"] = "Tu doanh"
df_new["ETL_date"] = str(dt.datetime.now())
df_new.head(5)

###########################

query = '''if (OBJECT_ID('fact_tu_doanh') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_tu_doanh
	(
		[Ma_ck] varchar(10),
        [Khoi_luong_mua_cp] float,
        [Khoi_luong_ban_cp] float,
        [Gia_tri_mua_trieu_vnd] float,
        [Gia_tri_ban_trieu_vnd] float,
        [Gia_tri_rong_trieu_vnd] float,
        [Khoi_luong_rong_cp] float,
		[Ngay] datetime2,
        [Phan_loai] nvarchar(20),
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

###############################

df_new.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_tu_doanh.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_tu_doanh.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_tu_doanh')
    cursor.executemany(''' INSERT INTO fact_tu_doanh Values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', reader)
    conn.commit()

df_tu_doanh = df_new.copy()

In [10]:
# ETL_data_4

path_text = f'D:\\Tin hoc\\Project\\Thong ke thi truong\\Market Update\\HSX\\Raw HSX\\GDTT\\GDTT'
path = pathlib.Path(path_text)

###################

lst_file = []
for direc in path.iterdir():
    for i in direc.iterdir():
        k = i.stem
        if not k.startswith("G"):
            lst_file.append(i)

########################

lst_df = []
for path in lst_file:
    skiprows = [i for i in range(0,7)]
    df = pd.read_excel(path, sheet_name='ExportData', skiprows=lambda x: x in skiprows)
    df = df.iloc[1:]
    df.columns = ["Ma_CK", "Ten", "Ngay_GD", "Gia_tri_thoa_thuan"]
    lst_df.append(df)
df_new = pd.DataFrame(pd.concat(lst_df))
df_new = pd.DataFrame(pd.concat(lst_df))
df_new = df_new[(df_new["Ten"].isnull() == False) & (df_new["Ngay_GD"].isnull() == False)]
df_new = df_new.reset_index(drop=True)
df_new["ETL_date"] = str(dt.datetime.now())
df_thoa_thuan = df_new.copy()

###########################

query = '''if (OBJECT_ID('fact_thoa_thuan') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_thoa_thuan
	(
        [Ma_ck] varchar(10),
        [Ten] nvarchar(100),
        [Ngay] datetime2,
        [Gia_tri_thoa_thuan_vnd] float,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

#############################

df_new.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_thoa_thuan.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_thoa_thuan.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_thoa_thuan')
    cursor.executemany(''' INSERT INTO fact_thoa_thuan Values (?, ?, ?, ?, ?)''', reader)
    conn.commit()

In [11]:
# ETL_data_5

df = pd.read_excel("Market HSX.xlsm", sheet_name = ["Khoi luong giao dich KL (xoay)", "Gia tri giao dich KL (xoay)", "So luong CP luu hanh (quy doi)", "Von hoa"])

#########################

df_klgdkl = df["Khoi luong giao dich KL (xoay)"]
lst_column = df_klgdkl.columns
lst_column = lst_column[1:]
lst_column_new = []
for i in lst_column:
    k = i.index(" ")
    ck = i[:k]
    lst_column_new.append(ck)
lst_column_new.insert(0, "Date")
df_klgdkl.columns = lst_column_new
df_klgdkl = df_klgdkl.melt(id_vars=["Date"], var_name="Ma_CK", value_name="KLGDKL")
df_klgdkl["ETL_date"] = str(dt.datetime.now())
df_klgdkl.head(2)

###########################

query = '''if (OBJECT_ID('fact_khoi_luong_gd_khop_lenh') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_khoi_luong_gd_khop_lenh
	(
        [Ngay] datetime2,
        [Ma_ck] varchar(10),
        [KLGDKL_cp] float,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

###########################

df_klgdkl.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_khoi_luong_gd_khop_lenh.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_khoi_luong_gd_khop_lenh.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_khoi_luong_gd_khop_lenh')
    cursor.executemany(''' INSERT INTO fact_khoi_luong_gd_khop_lenh Values (?, ?, ?, ?)''', reader)
    conn.commit()

#########################

df_gtgdkl = df["Gia tri giao dich KL (xoay)"]
lst_column = df_gtgdkl.columns
lst_column = lst_column[1:]
lst_column_new = []
for i in lst_column:
    k = i.index(" ")
    ck = i[:k]
    lst_column_new.append(ck)
lst_column_new.insert(0, "Date")
df_gtgdkl.columns = lst_column_new
df_gtgdkl = df_gtgdkl.melt(id_vars=["Date"], var_name="Ma_CK", value_name="GTGDKL")
df_gtgdkl["ETL_date"] = str(dt.datetime.now())
df_gtgdkl

############################

query = '''if (OBJECT_ID('fact_gia_tri_gd_khop_lenh') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_gia_tri_gd_khop_lenh
	(
        [Ngay] datetime2,
        [Ma_ck] varchar(10),
        [GTGDKL_vnd] float,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

####################################

df_gtgdkl.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_gia_tri_gd_khop_lenh.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_gia_tri_gd_khop_lenh.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_gia_tri_gd_khop_lenh')
    cursor.executemany(''' INSERT INTO fact_gia_tri_gd_khop_lenh Values (?, ?, ?, ?)''', reader)
    conn.commit()

###################################

df_slcplh = df["So luong CP luu hanh (quy doi)"]
lst_column = df_slcplh.columns
lst_column = lst_column[1:]
lst_column_new = []
for i in lst_column:
    k = i.index(" ")
    ck = i[:k]
    lst_column_new.append(ck)
lst_column_new.insert(0, "Date")
df_slcplh.columns = lst_column_new
df_slcplh = df_slcplh.melt(id_vars=["Date"], var_name="Ma_CK", value_name="SLCPLH")
df_slcplh["ETL_date"] = str(dt.datetime.now())
df_slcplh

###################################

query = '''if (OBJECT_ID('fact_so_luong_cp_luu_hanh') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_so_luong_cp_luu_hanh
	(
        [Ngay] datetime2,
        [Ma_ck] varchar(10),
        [SLCPLH_cp] float,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

###################################

df_slcplh.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_so_luong_cp_luu_hanh.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_so_luong_cp_luu_hanh.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_so_luong_cp_luu_hanh')
    cursor.executemany(''' INSERT INTO fact_so_luong_cp_luu_hanh Values (?, ?, ?, ?)''', reader)
    conn.commit()

##################################

df_vh = df["Von hoa"]
lst_column = df_vh.columns
lst_column = lst_column[1:]
lst_column_new = []
for i in lst_column:
    k = i.index(" ")
    ck = i[:k]
    lst_column_new.append(ck)
lst_column_new.insert(0, "Date")
df_vh.columns = lst_column_new
df_vh = df_vh.melt(id_vars=["Date"], var_name="Ma_CK", value_name="Von hoa")
df_vh["ETL_date"] = str(dt.datetime.now())
df_vh

##########################

query = '''if (OBJECT_ID('fact_von_hoa') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_von_hoa
	(
        [Ngay] datetime2,
        [Ma_ck] varchar(10),
        [Von_hoa_vnd] float,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

#################################

df_vh.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_von_hoa.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_von_hoa.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_von_hoa')
    cursor.executemany(''' INSERT INTO fact_von_hoa Values (?, ?, ?, ?)''', reader)
    conn.commit()


In [12]:
df_date = pd.DataFrame(list(df_vh["Date"].unique()))
df_date.columns = ["Date_join"]
df_date

,Date_join
0,2014-12-10
1,2014-12-11
2,2014-12-12
3,2014-12-15
4,2014-12-16
...,...
2125,2023-06-19
2126,2023-06-20
2127,2023-06-21
2128,2023-06-22


In [13]:
# ETL_data_3 

df = pd.read_excel('lay_gia.xlsx', sheet_name = "Lay Gia")
df = df.iloc[2:]

lst_column = list(df.iloc[0].values)
lst_column = lst_column[1:]
lst_column_new = []
for i in lst_column:
    index = i.index(' ')
    k = i[:index]
    lst_column_new.append(k)
lst_column_new.insert(0, "Date")

df.columns = lst_column_new
df = df.iloc[4:]
df = df.melt(id_vars=["Date"], var_name="Stock", value_name="Price")
df = df.reset_index(drop=True)
df["ETL_date"] = str(dt.datetime.now())

df = df.merge(df_date, how = "left", left_on=["Date"], right_on=["Date_join"])
df = df[df["Date_join"].isnull() == False]
df = df[["Date", "Stock", "Price", "ETL_date"]]
df_gia = df.copy()

#############################

query = '''if (OBJECT_ID('fact_gia') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_gia
	(
		[Ngay] datetime2,
        [Ma_ck] varchar(10),
        [Price] float,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

#################################

df.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_gia.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_gia.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_gia')
    cursor.executemany(''' INSERT INTO fact_gia Values (?, ?, ?, ?)''', reader)
    conn.commit()

################################

df_fl = pd.read_excel("Freefloat.xlsm", sheet_name = "Freefloat")
df_fl = df_fl.melt(id_vars=["Freefloat"], var_name="Stock", value_name="Freeloat1")
df_fl.columns = ["Date", "Stock", "Freefloat"]
df_fl = df_fl.reset_index(drop=True)
df_fl["ETL_date"] = str(dt.datetime.now())
df_fl.head(2) 

#################################

query = '''if (OBJECT_ID('fact_freefloat') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_freefloat
	(
		[Ngay] datetime2,
        [Ma_ck] varchar(10),
        [Freefloat_so_raw] float,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

##################################

df_fl.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_freefloat.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_freefloat.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_freefloat')
    cursor.executemany(''' INSERT INTO fact_freefloat Values (?, ?, ?, ?)''', reader)
    conn.commit()

In [14]:
df_date = pd.DataFrame(list(df_vh["Date"].unique()))
df_date.columns = ["Date_join"]
df_date

,Date_join
0,2014-12-10
1,2014-12-11
2,2014-12-12
3,2014-12-15
4,2014-12-16
...,...
2125,2023-06-19
2126,2023-06-20
2127,2023-06-21
2128,2023-06-22


In [16]:
df_gia = df_gia.merge(df_date, how = "left", left_on=["Date"], right_on=["Date_join"])
df_gia = df_gia[df_gia["Date_join"].isnull() == False]
df_gia = df_gia[["Date", "Stock", "Price", "ETL_date"]]

In [17]:
# ETL_data_6

lst_nam = ["2019", "2020", "2021", "2022"]
lst_thang = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]
lst_path_text = []
for i in lst_nam:
    for j in lst_thang:
        lst_path_text.append(f'D:\\Tin hoc\\Project\\Thong ke thi truong\\Market Update\\HSX\\Raw HSX\\Nuoc ngoai\\{i}\\Thang {j}')
lst_path_text = [pathlib.Path(i) for i in lst_path_text]

files = []
for path in lst_path_text:
    for direc in path.iterdir():
        file = direc.stem
        if not file.startswith("G"):
            files.append(direc)
files

####################

lst_df = []
for path in files:
    date_name = path.stem
    skiprows = [i for i in range(0,8)]
    df = pd.read_excel(path, sheet_name="ExportData", skiprows=lambda x: x in skiprows)
    df = df.iloc[2:, :-1]
    df["Ngay"] = dt.datetime.strptime(date_name, "%m.%d.%Y")
    lst_df.append(df)
df_new = pd.DataFrame(pd.concat(lst_df))
df_new = df_new[(df_new["Khối lượng mua(CP)"].isnull() == False) & (df_new["Khối lượng Bán(CP)"].isnull() == False)]
df_new = df_new.reset_index(drop = True)
df_new["ETL_Date"] = str(dt.datetime.now())
df_nuoc_ngoai = df_new.copy()
    
##########################

query = '''if (OBJECT_ID('fact_nuoc_ngoai') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_nuoc_ngoai
	(
        [Ma_ck] varchar(10),
        [Khoi_luong_mua_cp] float,
        [Gia_tri_mua_trieu_vnd] float,
        [Khoi_luong_ban_cp] float,
        [Gia_tri_ban_trieu_vnd] float,
        [Khoi_luong_rong_cp] float,
        [Gia_tri_rong_trieu_vnd] float,
        [Khoi_luong_khop_lenh_mua_cp] float,
        [Gia_tri_mua_khop_lenh_trieu_vnd] float,
        [Khoi_luong_ban_khop_lenh_cp] float,
        [Gia_tri_ban_khop_lenh_trieu_vnd] float,
        [Khoi_luong_thoa_thuan_mua_cp] float,
        [Gia_tri_mua_thoa_thuan_trieu_vnd] float,
        [Khoi_luong_ban_thoa_thuan_cp] float,
        [Gia_tri_ban_thoa_thuan_trieu_vnd] float,
        [Room_nuoc_ngoai_con_lai_cp] float,
        [Tong_room_nuoc_ngoai_duoc_phep_so_huu_cp] float,
        [%_tong_room_nuoc_ngoai_duoc_phep_so_huu] float,
        [Nuoc_ngoai_hien_dang_so_huu_cp] float,
        [%_nuoc_ngoai_so_huu] float,
        [Ngay] datetime2,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

#############################

df_new.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_nuoc_ngoai.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_nuoc_ngoai.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_nuoc_ngoai')
    cursor.executemany(''' INSERT INTO fact_nuoc_ngoai Values (?, ?, ?, ?, ?, ?,?, ?, ?, ?, ?, ?,?, ?, ?, ?, ?, ?,?, ?, ?, ?)''', reader)
    conn.commit()

In [19]:
# ETL_data_8

lst_thang = ["1", "2", "3", "4", "5", "6"]
lst_path_text = []
for j in lst_thang:
    lst_path_text.append(f'D:\\Tin hoc\\Project\\Thong ke thi truong\\Market Update\\HSX\\Raw HSX\\thong ke dat lenh\\thong ke dat lenh\\2022\\T{j}')
lst_path_text = [pathlib.Path(i) for i in lst_path_text]

files = []
for path in lst_path_text:
    for direc in path.iterdir():
        file = direc.stem
        if not file.startswith("G"):
            files.append(direc)
files

######################

lst_df = []
for path in files:
    date_name = path.stem
    skiprows = [i for i in range(0,11)]
    df = pd.read_excel(path, sheet_name="ExportData", skiprows=lambda x: x in skiprows)
    df = df.iloc[2:, :-1]
    df["Ngay"] = dt.datetime.strptime(date_name, "%m.%d.%Y")
    lst_df.append(df)

df_new = pd.DataFrame(pd.concat(lst_df))
df_new = df_new[(df_new["Dư mua(CP)"].isnull() == False) & (df_new["Dư bán(CP)"].isnull() == False)]
df_new = df_new.reset_index(drop=True)
df_new["ETL_date"] = str(dt.datetime.now())
df_thong_ke_dat_lenh = df_new.copy()
################################

query = '''if (OBJECT_ID('fact_thong_ke_dat_lenh') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_thong_ke_dat_lenh
	(
        [Ma_ck] varchar(10),
        [Du_mua_cp] float,
        [Du_ban_cp] float,
        [So_lenh_mua_lenh] float,
        [Khoi_luong_mua_cp] float,
        [Trung_binh_lenh_mua] float,
        [So_lenh_ban] float,
        [Khoi_luong_ban_cp] float,
        [Trung_binh_lenh_ban] float,
        [Chenh_lech_khoi_luong_dat_mua_dat_ban] float,
        [Ngay] datetime2,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

#################################

df_new.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_thong_ke_dat_lenh.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_thong_ke_dat_lenh.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_thong_ke_dat_lenh')
    cursor.executemany(''' INSERT INTO fact_thong_ke_dat_lenh Values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', reader)
    conn.commit()

In [20]:
# ETL_data_9 
df = pd.read_excel("VNItong.xlsm")
df = df.iloc[:, :-5]
df = df[~(df["Ngày"].dt.year == 2023)]
df = df.reset_index(drop=True)
df["ETL_date"] = str(dt.datetime.now())
df

##########################

query = '''if (OBJECT_ID('fact_vni') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_vni
	(
        [Ngay] datetime2,
        [Thay_doi] float,
        [Mo_cua] float,
        [Cao_nhat] float,
        [Thap_nhat] float,
        [Dong_cua] float,
        [Khop_lenh_khoi_luong_cp] float,
        [Gia_tri_khop_lenh_ty_vnd] float,
        [Thoa_thuan_khoi_luong_cp] float,
        [Gia_tri_thoa_thuan_ty_vnd] float,
        [KLGD_cp] float,
        [GTGD_ty_vnd] float,
        [Von_hoa_ty_vnd] float,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

###########################

df.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_vni.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_vni.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_vni')
    cursor.executemany(''' INSERT INTO fact_vni Values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', reader)
    conn.commit()

In [21]:
# ETL_data_10

lst_nam = ["2019", "2020", "2021", "2022"]
lst_thang = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]
lst_path_text = []
for i in lst_nam:
    for j in lst_thang:
        lst_path_text.append(f'D:\\Tin hoc\\Project\\Thong ke thi truong\\Market Update\\HSX\\Raw HSX\\VNI chi tiet\\{i}\\Thang {j}')
lst_path_text = [pathlib.Path(i) for i in lst_path_text]

files = []
for path in lst_path_text:
    for direc in path.iterdir():
        file = direc.stem
        if not file.startswith("G"):
            files.append(direc)
files

########################################

lst_df = []
for path in files:
    date_name = path.stem
    skiprows = [i for i in range(0,8)]
    df = pd.read_excel(path, sheet_name="ExportData", skiprows=lambda x: x in skiprows)
    df = df.iloc[:, :-1]
    lst_columns = ["Ma_ck", "Thay_doi(+/-%)", "Mo_cua", "Cao_nhat", "Thap_nhat", "Dong_cua", "Khop_lenh_khoi_luong", "Khop_lenh_gia_tri",
                "Thoa_thuan_khoi_luong", "Thoa_thuan_gia_tri", "KLGD(CP)", "GTGD(trieu_VND)", "Von_hoa(trieu_VND)"]
    df.columns = lst_columns
    df["Ngay"] = dt.datetime.strptime(date_name, "%m.%d.%Y")
    df = df.iloc[3:]
    lst_df.append(df)

df_new = pd.DataFrame(pd.concat(lst_df))
df_new["ETL_date"] = str(dt.datetime.now())
df_new = df_new[(df_new["Mo_cua"].isnull() == False) & (df_new["Cao_nhat"].isnull() == False)]
df_new = df_new.reset_index(drop=True)
df_new.fillna(value=0)
df_new["Khop_lenh_khoi_luong"] = df_new["Khop_lenh_khoi_luong"].astype(float)
df_new["Khop_lenh_gia_tri"] = df_new["Khop_lenh_gia_tri"].astype(float)
df_new["Thoa_thuan_khoi_luong"] = df_new["Thoa_thuan_khoi_luong"].astype(float)
df_new["Thoa_thuan_gia_tri"] = df_new["Thoa_thuan_gia_tri"].astype(float)
df_vni_chi_tiet = df_new.copy()

##########################################

query = '''if (OBJECT_ID('fact_vni_chi_tiet') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_vni_chi_tiet
	(
        [Ma_ck] varchar(10),
        [Thay_doi(+/-%)] float,
        [Mo_cua] float,
        [Cao_nhat] float,
        [Thap_nhat] float,
        [Dong_cua] float,
        [Khop_lenh_khoi_luong_cp] float,
        [Khop_lenh_gia_tri_trieu_vnd] float,
        [Thoa_thuan_khoi_luong_cp] float,
        [Thoa_thuan_gia_tri_trieu_vnd] float,
        [KLGD(CP)] float,
        [GTGD(trieu_VND)] float,
        [Von_hoa(trieu_VND)] float,
        [Ngay] datetime2,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

###########################################

df_new.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_vni_chi_tiet.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_vni_chi_tiet.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_vni_chi_tiet')
    cursor.executemany(''' INSERT INTO fact_vni_chi_tiet Values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', reader)
    conn.commit()

C:\Users\Admin\AppData\Local\Temp\ipykernel_6880\3474162970.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_new.fillna(value=0)


In [22]:
# ETL_data_11

path_text = r'D:\Tin hoc\Project\Thong ke thi truong\Market Update\HSX\Raw HSX\Chi_tiet_lich_su_gia_Index'
path = pathlib.Path(path_text)
lst = os.listdir(path)

lst_path_new = []
for i in lst:
    path_new = path/i 
    lst_path_new.append(path_new)

file = []
for direc in lst_path_new:
    for i in direc.iterdir():
        k = i.suffix
        if k == '.csv':
            file.append(i)

##############################

lst_df = []
for path in file:
    type_name = path.stem 
    df = pd.read_csv(path)
    df["type"] = type_name 
    lst_df.append(df)

df_new = pd.DataFrame(pd.concat(lst_df))
df_new = df_new.reset_index(drop=True)
df_new["ETL_date"] = str(dt.datetime.now())
df_new["Vol."] = df_new["Vol."].fillna(0)
df_new["Vol_new"] = df_new["Vol."].str.slice(0,-1)
df_new["Vol_new"] = df_new["Vol_new"].astype(str)
df_new['Vol_new'] = df_new['Vol_new'].replace(np.nan, '0')
df_new['abc'] = df_new["Vol."].str.slice(-1)
df_new['abc'] = df_new['abc'].fillna('-')

lst = list(df_new["Vol_new"].values)
lst_new = []
for i in lst:
    g = i.find(',')
    if g == -1:
        lst_new.append(i)
    else:
        k = i.replace(',','_')
        lst_new.append(k)
lst_new = [0 if i == '' else i for i in lst_new]
lst_new = [float(i) for i in lst_new]
df_lst_new = pd.DataFrame(lst_new)
df_new['Vol_new'] = df_lst_new
df_new['Vol_new'] = df_new['Vol_new'].fillna(0)

def phan_loai_1(row):
    last_word = row.loc["abc"]
    vol_new = row.loc["Vol_new"]

    if last_word == 'B':
        return vol_new * 1000000000
        
    elif last_word == 'K':
        return vol_new * 1000
        
    elif last_word == 'M':
        return vol_new * 1000000

    else: return vol_new

df_new['Vol_new'] = df_new.apply(phan_loai_1, axis="columns")
df_new = df_new[["Date", "Price", "Open", "High", "Low", "Vol_new", "type", "ETL_date"]]
df_new["Date"] = df_new["Date"].astype("datetime64[ns]")

lst_ten = ["Price", "Open", "High", "Low"]
for t in lst_ten:
    df_new[t] = df_new[t].astype(str)
    lst = list(df_new[t].values)
    lst_new = []
    for i in lst:
        g = i.find(',')
        if g == -1:
            lst_new.append(i)
        else:
            k = i.replace(',','_')
            lst_new.append(k)
    lst_new = [0 if i == '' else i for i in lst_new]
    lst_new = [float(i) for i in lst_new]
    df_lst_new = pd.DataFrame(lst_new)
    df_new[t] = df_lst_new
    df_new[t] = df_new[t].fillna(0)
    df_new[t] = df_new[t].astype(float)
df_new

##################################

query = '''if (OBJECT_ID('fact_index') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_index
	(
		[Date] datetime2,
		[Price] float,
        [Open] float,
        [High] float,
        [Low] float,
        [Vol_new] float,
        [type] varchar(50),
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

##############################

df_new.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_index.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_index.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_index')
    cursor.executemany(''' INSERT INTO fact_index Values (?, ?, ?, ?, ?, ?, ?, ?)''', reader)
    conn.commit()

In [23]:
# ETL_data_12

lst_date_gia = list(df_gia["Date"].unique())
df_date_gia = pd.DataFrame(lst_date_gia)
df_date_gia.columns = ["date_gia"]
df_date_gia

lst = pd.date_range(start="2014-12-11", end="2022-12-31", freq="D")
df = pd.DataFrame(lst)
df.columns = ["date"]
df["week_day"] = df["date"].dt.weekday
df = df.merge(df_date_gia, how='left', left_on='date', right_on='date_gia')
df["date_gia"] = df["date_gia"].astype(str)
# df["date_id"] = df.apply(lambda row: if row["week_day"], axis=1)

lst_day = [int(i) for i in list(df["week_day"].values)]
lst_date_gia = list(df["date_gia"].values)
count = 0
lst_date_id = []
for j,i in enumerate(lst_day): 
    if i in (0,1,2,3,4) and lst_date_gia[j] != 'NaT':
        count = count + 1
        lst_date_id.append(count)
    else:
        lst_date_id.append(0)

df["date_id"] = pd.DataFrame(lst_date_id)
df["ETL_date"] = str(dt.datetime.now())
df = df[["date", "week_day", "date_id", "ETL_date"]]

#################################

query = '''if (OBJECT_ID('dim_date_id') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_date_id
	(
		[date] datetime2,
		[week_day] int,
        [date_id] int,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

#####################################

df.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_date_id.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_date_id.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_date_id')
    cursor.executemany(''' INSERT INTO dim_date_id Values (?, ?, ?, ?)''', reader)
    conn.commit()

#####################################

path_text = r'D:\Tin hoc\Project\Thong ke thi truong\Market Update\HSX\Du no margin'
path = pathlib.Path(path_text)
lst = os.listdir(path)

lst_path_new = []
for i in lst:
    path_new = path/i 
    lst_path_new.append(path_new)

lst_path_new

####################################

lst_df = []
for path in lst_path_new:
    date_name = path.stem
    skiprows = [i for i in range(0,11)]
    df = pd.read_excel(path, sheet_name="ExportData", skiprows=lambda x: x in skiprows)

    df1 = pd.DataFrame(df.iloc[0])
    df1.columns = ['cot']
    df1 = df1.reset_index()

    lst = list(df1['index'].values)
    for i,j in enumerate(lst):
        if j.startswith('U'):
            k = lst[i-1]
            lst[i] = k 
    df1['index'] =  pd.DataFrame(lst)
    df1 = df1.iloc[1:]
    df1['column'] = df1['index'].str.slice(0,2,1) + ' - ' + df1['cot'] 
    lst_column = list(df1['column'].values)
    lst_column.insert(0, 'Cong ty chung khoan')

    df.columns = lst_column
    df = df.iloc[2:]
    df["Nam"] = date_name
    df["Nam"] = df["Nam"].astype(int)

    lst_df.append(df)

df_new = pd.DataFrame(pd.concat(lst_df))
df_new["ETL_date"] = str(dt.datetime.now())
df_new = df_new.reset_index(drop = True)
df_new = df_new[(df_new["Q1 - Nợ vay ký quỹ (Tỷ VND)"].isnull() == False) & (df_new["Q1 - Nợ vay ký quỹ/ Vốn chủ sở hữu (%)"].isnull() == False)]
df_new["Cong ty chung khoan"] = df_new["Cong ty chung khoan"].fillna('null')

lst_column1 = list(df_new.columns)
lst_column1 = lst_column1[1:-2]

for i in lst_column1:
    df_new[i] = df_new[i].fillna(0)
    df_new[i] = df_new[i].replace('-', 0)
    df_new[i] = df_new[i].astype(float)

df_new

##############################

query = '''if (OBJECT_ID('fact_margin') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_margin
	(
		[Cong ty chung khoan] nvarchar(100),
		[Q1_No_vay_ky_quy(Ty_VND)] float,
        [Q1_No_vay_ky_quy_/_Von_chu_so_huu_(%)] float,
        [Q1_No_vay_ky_quy_/_Tong_tai_san_(%)] float,
        [Q2_No_vay_ky_quy_(Ty_VND)] float,
        [Q2_No_vay_ky_quy_/_von_chu_so_huu_(%)] float,
        [Q2_No_vay_ky_quy_/_Tong_tai_san_(%)] float,
        [Q3_No_vay_ky_quy_(Ty_VND)] float,
        [Q3_No_vay_ky_quy_/_von_chu_so_huu_(%)] float,
        [Q3_No_vay_ky_quy_/_Tong_tai_san_(%)] float,
        [Q4_No_vay_ky_quy_(Ty_VND)] float,
        [Q4_No_vay_ky_quy_/_von_chu_so_huu_(%)] float,
        [Q4_No_vay_ky_quy_/_Tong_tai_san_(%)] float,
        [Nam] int,
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

#################################

df_new.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_margin.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\fact_margin.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_margin')
    cursor.executemany(''' INSERT INTO fact_margin Values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', reader)
    conn.commit()

C:\Users\Admin\AppData\Local\Temp\ipykernel_6880\3412205637.py:113: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_new[i] = df_new[i].fillna(0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_6880\3412205637.py:113: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_new[i] = df_new[i].fillna(0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_6880\3412205637.py:113: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future beha

In [24]:
df_investor_group = pd.DataFrame(list(set(lst_phan_nhom)), columns=["investor"])
df_investor_group.loc[-1] = [df_tu_doanh["phan_loai"].unique()[0]]
df_investor_group["ETL_date"] = str(dt.datetime.now())
df_investor_group = df_investor_group.reset_index(drop=True)

query = '''if (OBJECT_ID('dim_investor') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_investor
	(
		[investor] nvarchar(100),
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

df_investor_group.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_investor.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_investor.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_investor')
    cursor.executemany(''' INSERT INTO dim_investor Values (?, ?)''', reader)
    conn.commit()

In [ ]:
# ETL_data_7 

skiprows = [i for i in range(0,7)]
df = pd.read_excel("Look_up_nganh.xlsm", sheet_name="ExportData", skiprows=lambda x: x in skiprows)
df = df.iloc[1:453, 1:]
df = df.reset_index(drop=True)
df.columns = ["Ma_ck", "Ten", "San", "Lv1", "Lv2", "Lv3", "Lv4", "Lv5"]
df 

#################

df_phan_nganh = df[["Ma_ck", "Lv1", "Lv2", "Lv3", "Lv4", "Lv5"]].copy()
lst_index = list(df_phan_nganh.index)
df_phan_nganh["id_stock"] = pd.DataFrame(lst_index)
df_phan_nganh

#################

lst_lv1 = df["Lv1"].unique()
df_lv1 = pd.DataFrame(lst_lv1)
df_lv1.columns = ["phan_nganh_lv1"]
lst_index_lv1 = list(df_lv1.index)
df_lv1["id_lv1"] = pd.DataFrame(lst_index_lv1)
df_lv1["ETL_date"] = str(dt.datetime.now())
df_lv1

###################

lst_lv2 = df["Lv2"].unique()
df_lv2 = pd.DataFrame(lst_lv2)
df_lv2.columns = ["phan_nganh_lv2"]
lst_index_lv2 = list(df_lv2.index)
df_lv2["id_lv2"] = pd.DataFrame(lst_index_lv2)
# df_lv2.loc[-1] = ["Chưa phân ngành", -1]
df_lv2["ETL_date"] = str(dt.datetime.now())
df_lv2

##################################

lst_lv3 = df["Lv3"].unique()
df_lv3 = pd.DataFrame(lst_lv3)
df_lv3.columns = ["phan_nganh_lv3"]
lst_index_lv3 = list(df_lv3.index)
df_lv3["id_lv3"] = pd.DataFrame(lst_index_lv3)
# df_lv3.loc[-1] = ["Chưa phân ngành", -1]
df_lv3["ETL_date"] = str(dt.datetime.now())
df_lv3

##################################

lst_lv4 = df["Lv4"].unique()
df_lv4 = pd.DataFrame(lst_lv4)
df_lv4.columns = ["phan_nganh_lv4"]
lst_index_lv4 = list(df_lv4.index)
df_lv4["id_lv4"] = pd.DataFrame(lst_index_lv4)
# df_lv4.loc[-1] = ["Chưa phân ngành", -1]
df_lv4["ETL_date"] = str(dt.datetime.now())
df_lv4

####################################

lst_lv5 = df["Lv5"].unique()
df_lv5 = pd.DataFrame(lst_lv5)
df_lv5.columns = ["phan_nganh_lv5"]
lst_index_lv5 = list(df_lv5.index)
df_lv5["id_lv5"] = pd.DataFrame(lst_index_lv5)
# df_lv5.loc[-1] = ["Chưa phân ngành", -1]
df_lv5["ETL_date"] = str(dt.datetime.now())
df_lv5

###########################################

df_phan_nganh = df_phan_nganh.merge(right=df_lv1, how='left', left_on='Lv1', right_on="phan_nganh_lv1").drop(columns=["phan_nganh_lv1", "Lv1", "ETL_date"], axis=1)
df_phan_nganh = df_phan_nganh.merge(right=df_lv2, how='left', left_on='Lv2', right_on="phan_nganh_lv2").drop(columns=["phan_nganh_lv2", "Lv2", "ETL_date"], axis=1)
df_phan_nganh = df_phan_nganh.merge(right=df_lv3, how='left', left_on='Lv3', right_on="phan_nganh_lv3").drop(columns=["phan_nganh_lv3", "Lv3", "ETL_date"], axis=1)
df_phan_nganh = df_phan_nganh.merge(right=df_lv4, how='left', left_on='Lv4', right_on="phan_nganh_lv4").drop(columns=["phan_nganh_lv4", "Lv4", "ETL_date"], axis=1)
df_phan_nganh = df_phan_nganh.merge(right=df_lv5, how='left', left_on='Lv5', right_on="phan_nganh_lv5").drop(columns=["phan_nganh_lv5", "Lv5", "ETL_date"], axis=1)
df_phan_nganh = df_phan_nganh.fillna(-1)
df_phan_nganh["id_stock"] = df_phan_nganh["id_stock"].astype(int)
df_phan_nganh["id_lv1"] = df_phan_nganh["id_lv1"].astype(str) + '.1'
df_phan_nganh["id_lv2"] = df_phan_nganh["id_lv2"].astype(str) + '.2'
df_phan_nganh["id_lv3"] = df_phan_nganh["id_lv3"].astype(str) + '.3'
df_phan_nganh["id_lv4"] = df_phan_nganh["id_lv4"].astype(str) + '.4'
df_phan_nganh["id_lv5"] = df_phan_nganh["id_lv5"].astype(str) + '.5'
df_phan_nganh.columns = ["Ma_ck", "id_stock", "Cấp 1", "Cấp 2", "Cấp 3", "Cấp 4", "Cấp 5"]
df_phan_nganh = df_phan_nganh.melt(id_vars=["Ma_ck", "id_stock"], var_name="Phan_loai", value_name="id_phan_nganh")
df_phan_nganh["ETL_date"] = str(dt.datetime.now())

#################################

query = '''if (OBJECT_ID('dim_phan_nganh') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_phan_nganh
	(
        [Ma_ck] varchar(10),
        [id_stock] int,
        [phan_loai] nvarchar(20), 
        [id_phan_nganh] varchar(10),
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

#########################

df_phan_nganh.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_phan_nganh.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_phan_nganh.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_phan_nganh')
    cursor.executemany(''' INSERT INTO dim_phan_nganh Values (?, ?, ?, ?, ?)''', reader)
    conn.commit()

###########################################

lst_lv1 = df["Lv1"].unique()
df_lv1 = pd.DataFrame(lst_lv1)
df_lv1.columns = ["ten_nganh"]
lst_index_lv1 = list(df_lv1.index)
df_lv1["id"] = pd.DataFrame(lst_index_lv1)
df_lv1["id"] = df_lv1["id"].astype(str)
df_lv1["id"] = df_lv1["id"] + '.1'
df_lv1["phan_nganh"] = "Cấp 1"
df_lv1["ETL_date"] = str(dt.datetime.now())
df_lv1.head(2)

query = '''if (OBJECT_ID('dim_nganh_lv1') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_nganh_lv1
	(
        [ten_nganh] nvarchar(100),
        [id] varchar(10),
        [phan_nganh] nvarchar(20),
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

df_lv1.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_nganh_lv1.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_nganh_lv1.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_nganh_lv1')
    cursor.executemany(''' INSERT INTO dim_nganh_lv1 Values (?, ?, ?, ?)''', reader)
    conn.commit()

##################################################

lst_lv2 = df["Lv2"].unique()
df_lv2 = pd.DataFrame(lst_lv2)
df_lv2.columns = ["ten_nganh"]
lst_index_lv2 = list(df_lv2.index)
df_lv2["id"] = pd.DataFrame(lst_index_lv2)
df_lv2["id"] = df_lv2["id"].astype(str)
df_lv2["id"] = df_lv2["id"] + '.2'
df_lv2["phan_nganh"] = "Cấp 2"
df_lv2["ETL_date"] = str(dt.datetime.now())
df_lv2.head(2)

query = '''if (OBJECT_ID('dim_nganh_lv2') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_nganh_lv2
	(
        [ten_nganh] nvarchar(100),
        [id] varchar(10),
        [phan_nganh] nvarchar(20),
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

df_lv2.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_nganh_lv2.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_nganh_lv2.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_nganh_lv2')
    cursor.executemany(''' INSERT INTO dim_nganh_lv2 Values (?, ?, ?, ?)''', reader)
    conn.commit()

##############################################

lst_lv3 = df["Lv3"].unique()
df_lv3 = pd.DataFrame(lst_lv3)
df_lv3.columns = ["ten_nganh"]
lst_index_lv3 = list(df_lv3.index)
df_lv3["id"] = pd.DataFrame(lst_index_lv3)
df_lv3["id"] = df_lv3["id"].astype(str)
df_lv3["id"] = df_lv3["id"] + '.3'
df_lv3["phan_nganh"] = "Cấp 3"
df_lv3["ETL_date"] = str(dt.datetime.now())
df_lv3.head(2)

query = '''if (OBJECT_ID('dim_nganh_lv3') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_nganh_lv3
	(
        [ten_nganh] nvarchar(100),
        [id] varchar(10),
        [phan_nganh] nvarchar(20),
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

df_lv3.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_nganh_lv3.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_nganh_lv3.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_nganh_lv3')
    cursor.executemany(''' INSERT INTO dim_nganh_lv3 Values (?, ?, ?, ?)''', reader)
    conn.commit()

######################################################

lst_lv4 = df["Lv4"].unique()
df_lv4 = pd.DataFrame(lst_lv4)
df_lv4.columns = ["ten_nganh"]
lst_index_lv4 = list(df_lv4.index)
df_lv4["id"] = pd.DataFrame(lst_index_lv4)
df_lv4["id"] = df_lv4["id"].astype(str)
df_lv4["id"] = df_lv4["id"] + '.4'
df_lv4["phan_nganh"] = "Cấp 4"
df_lv4["ETL_date"] = str(dt.datetime.now())
df_lv4.head(2)

query = '''if (OBJECT_ID('dim_nganh_lv4') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_nganh_lv4
	(
        [ten_nganh] nvarchar(100),
        [id] varchar(10),
        [phan_nganh] nvarchar(20),
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

df_lv4.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_nganh_lv4.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_nganh_lv4.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_nganh_lv4')
    cursor.executemany(''' INSERT INTO dim_nganh_lv4 Values (?, ?, ?, ?)''', reader)
    conn.commit()

##########################################################

lst_lv5 = df["Lv5"].unique()
df_lv5 = pd.DataFrame(lst_lv5)
df_lv5.columns = ["ten_nganh"]
lst_index_lv5 = list(df_lv5.index)
df_lv5["id"] = pd.DataFrame(lst_index_lv5)
df_lv5["id"] = df_lv5["id"].astype(str)
df_lv5["id"] = df_lv5["id"] + '.5'
df_lv5["phan_nganh"] = "Cấp 5"
df_lv5["ETL_date"] = str(dt.datetime.now())
df_lv5.head(2)

query = '''if (OBJECT_ID('dim_nganh_lv5') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_nganh_lv5
	(
        [ten_nganh] nvarchar(100),
        [id] varchar(10),
        [phan_nganh] nvarchar(20),
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

df_lv5.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_nganh_lv5.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_nganh_lv5.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_nganh_lv5')
    cursor.executemany(''' INSERT INTO dim_nganh_lv5 Values (?, ?, ?, ?)''', reader)
    conn.commit()

####################################################################################

df_phan_nganh_cac_cap = pd.concat([df_lv1, df_lv2, df_lv3, df_lv4, df_lv5])
df_phan_nganh_cac_cap = df_phan_nganh_cac_cap.reset_index(drop=True)
df_phan_nganh_cac_cap["ETL_date"] = str(dt.datetime.now())
df_phan_nganh_cac_cap

query = '''if (OBJECT_ID('dim_nganh_cac_cap') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_nganh_cac_cap
	(
        [ten_nganh] nvarchar(100),
        [id] varchar(10),
        [phan_nganh] nvarchar(20),
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

df_phan_nganh_cac_cap.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_nganh_cac_cap.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_nganh_cac_cap.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_nganh_cac_cap')
    cursor.executemany(''' INSERT INTO dim_nganh_cac_cap Values (?, ?, ?, ?)''', reader)
    conn.commit()
    
##########################################

lst_phan_cap = ["Cấp 1", "Cấp 2", "Cấp 3", "Cấp 4", "Cấp 5"]
df_phan_cap = pd.DataFrame(lst_phan_cap)
df_phan_cap.columns = ["cap_nganh"]
df_phan_cap["ETL_date"] = str(dt.datetime.now())

query = '''if (OBJECT_ID('dim_cap_nganh') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_cap_nganh
	(
        [cap_nganh] nvarchar(20),
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

df_phan_cap.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_cap_nganh.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_cap_nganh.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_cap_nganh')
    cursor.executemany(''' INSERT INTO dim_cap_nganh Values (?, ?)''', reader)
    conn.commit()

In [26]:
lst_ck_gtm = list(df_gtm["Ma_ck"].unique())

In [27]:
lst_ck_klm = list(df_klm["Ma_ck"].unique())

In [28]:
lst_ck_gtb = list(df_gtb["Ma_ck"].unique())

In [29]:
lst_ck_klb = list(df_klb["Ma_ck"].unique())

In [30]:
lst_ck_gtr = list(df_gtr["Ma_ck"].unique())

In [31]:
lst_ck_klr = list(df_klr["Ma_ck"].unique())

In [32]:
# df_tu_doanh
lst_ck_tu_doanh = list(df_tu_doanh["Mã CK"].unique())

In [33]:
# df_thoa_thuan
lst_ck_thoa_thuan = list(df_thoa_thuan["Ma_CK"].unique())

In [34]:
# df_gtgdkl
lst_ck_gtgdkl = list(df_gtgdkl["Ma_CK"].unique())

In [35]:
# df_klgdkl
lst_ck_klgdkl = list(df_klgdkl["Ma_CK"].unique())

In [36]:
# df_slcplh
lst_ck_slcplh = list(df_slcplh["Ma_CK"].unique())

In [37]:
# df_vh
lst_ck_vh = list(df_vh["Ma_CK"].unique())

In [38]:
# df_gia
lst_ck_gia = list(df_gia["Stock"].unique())

In [39]:
# df_fl
lst_ck_fl = list(df_fl["Stock"].unique())

In [40]:
# df_nuoc_ngoai
lst_ck_nuoc_ngoai = list(df_nuoc_ngoai["Mã CK"].unique())

In [41]:
# df_phan_nganh
lst_ck_phan_nganh = list(df_phan_nganh["Ma_ck"].unique())

In [42]:
# df_thong_ke_dat_lenh
lst_ck_thong_ke_dat_lenh = list(df_thong_ke_dat_lenh["Mã CK"].unique())

In [43]:
# df_vni_chi_tiet
lst_ck_vni_chi_tiet = list(df_vni_chi_tiet["Ma_ck"].unique())

In [44]:
lst_ck_unique = lst_ck_gtm + lst_ck_klm + lst_ck_gtb + lst_ck_klb + lst_ck_gtr + lst_ck_klr + lst_ck_tu_doanh \
                + lst_ck_thoa_thuan + lst_ck_gtgdkl + lst_ck_klgdkl + lst_ck_slcplh + lst_ck_vh + lst_ck_gia  \
                + lst_ck_fl + lst_ck_nuoc_ngoai + lst_ck_phan_nganh + lst_ck_thong_ke_dat_lenh + lst_ck_vni_chi_tiet

In [45]:
lst_ck_unique = list(set(lst_ck_unique))
df_ck_unique = pd.DataFrame(lst_ck_unique)
df_ck_unique.columns = ["Ma_Ck_unique"]
df_ck_unique

,Ma_Ck_unique
0,KMR
1,THI
2,DGC
3,SCD
4,PTC
...,...
447,CMX
448,EIB
449,TDW
450,HSL


In [106]:
df_phan_nganh_pivot = df_phan_nganh.pivot(columns=["Phan_loai"], values=["id_phan_nganh"], index=["Ma_ck", "id_stock"])
df_phan_nganh_pivot.columns = ["id_lv1", "id_lv2", "id_lv3", "id_lv4", "id_lv5"]
df_phan_nganh_pivot = df_phan_nganh_pivot.reset_index(drop=False)

df_stock = pd.read_csv(r'D:\Tin hoc\Project\Thong ke thi truong\Market Update\HSX\Stock\stock.csv')
df_stock = df_stock.merge(right = df_phan_nganh_pivot, how = 'left', left_on='Ma_ck', right_on="Ma_ck")
df_stock = df_stock[["Stock Id", "Ma_ck", "Ten_doanh_nghiep", "id_lv1", "id_lv2", "id_lv3", "id_lv4", "id_lv5"]]
df_stock = df_stock.fillna(-1)
df_stock = df_stock.merge(df_ck_unique, how = 'right', left_on='Ma_ck', right_on='Ma_Ck_unique')
df_stock["Stock Id"] = df_stock["Stock Id"].fillna(-1)
df_stock["Stock Id"] = df_stock["Stock Id"].astype(int)
df_stock["id_lv1"] = df_stock["id_lv1"].fillna(-1)
df_stock["id_lv2"] = df_stock["id_lv2"].fillna(-1)
df_stock["id_lv3"] = df_stock["id_lv3"].fillna(-1)
df_stock["id_lv4"] = df_stock["id_lv4"].fillna(-1)
df_stock["id_lv5"] = df_stock["id_lv5"].fillna(-1)
df_stock["Ten_doanh_nghiep"] = df_stock["Ten_doanh_nghiep"].fillna('Blank')
df_stock = df_stock[["Stock Id", "Ma_Ck_unique", "Ten_doanh_nghiep", "id_lv1", "id_lv2", "id_lv3", "id_lv4", "id_lv5"]]
df_stock.columns = ["Stock Id", "Ma_ck", "Ten_doanh_nghiep", "id_lv1", "id_lv2", "id_lv3", "id_lv4", "id_lv5"]

df_stock["ETL_date"] = str(dt.datetime.now())
df_stock = df_stock.merge(df_phan_nganh_cac_cap, how = 'left', left_on='id_lv1', right_on='id').drop(columns=['ETL_date_x', 'id', 'phan_nganh', 'ETL_date_y']).rename(columns={'ten_nganh': 'ten_nganh_lv1'})
df_stock = df_stock.merge(df_phan_nganh_cac_cap, how = 'left', left_on='id_lv2', right_on='id').drop(columns=['ETL_date', 'id', 'phan_nganh']).rename(columns={'ten_nganh': 'ten_nganh_lv2'})
df_stock = df_stock.merge(df_phan_nganh_cac_cap, how = 'left', left_on='id_lv3', right_on='id').drop(columns=['ETL_date', 'id', 'phan_nganh']).rename(columns={'ten_nganh': 'ten_nganh_lv3'})
df_stock = df_stock.merge(df_phan_nganh_cac_cap, how = 'left', left_on='id_lv4', right_on='id').drop(columns=['ETL_date', 'id', 'phan_nganh']).rename(columns={'ten_nganh': 'ten_nganh_lv4'})
df_stock = df_stock.merge(df_phan_nganh_cac_cap, how = 'left', left_on='id_lv5', right_on='id').drop(columns=['ETL_date', 'id', 'phan_nganh']).rename(columns={'ten_nganh': 'ten_nganh_lv5'})
df_stock = df_stock.fillna('Chưa phân ngành')
df_stock["ETL_date"] = str(dt.datetime.now())

query = '''if (OBJECT_ID('dim_stock') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_stock
	(
		[Stock Id] int,
		[Ma_ck] varchar(10),
        [Ten_doanh_nghiep] nvarchar(100),
        [id_lv1] float,
        [id_lv2] float,
        [id_lv3] float,
        [id_lv4] float,
        [id_lv5] float,
        [ten_nganh_lv1] nvarchar(100),
        [ten_nganh_lv2] nvarchar(100),
        [ten_nganh_lv3] nvarchar(100),
        [ten_nganh_lv4] nvarchar(100),
        [ten_nganh_lv5] nvarchar(100),
        [ETL_date] varchar(50)
	);
end'''
cursor.execute(query)
conn.commit()

df_stock.to_csv(f"D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_stock.txt", index=False)
with open("D:\\Tin hoc\\Project\\Thong ke thi truong\\database_file\\dim_stock.txt", newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_stock')
    cursor.executemany(''' INSERT INTO dim_stock Values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', reader)
    conn.commit()